In [25]:
# Create a SnpReader for reading PLINK Bed files (local or network).
# Look at the metadata without reading the genotype data.
from pysnptools.snpreader import Bed
url = "https://raw.githubusercontent.com/fastlmm/FaST-LMM/refs/heads/master/tests/datasets/all_chr.maf0.001.N300.bed"
snp_reader = Bed(url, count_A1=True)
print(f"Shape: {snp_reader.shape} (individuals × SNPs)")

Shape: (300, 1015) (individuals × SNPs)


In [26]:
import numpy as np

# List the first 5 individual (sample) ids, the first 5 SNP (variant) ids,
# and every unique chromosome.
print(snp_reader.iid[:5])
print(snp_reader.sid[:5])
print(np.unique(snp_reader.pos[:,0])) # "pos" is chrom, bp_position, and cm_position

# Read all genotype data over network to an in-memory SnpData
# that wraps a numpy array.
snp_data_all = snp_reader.read()
display(snp_data_all.val)

# Create a new SnpReader for every second individual and SNPs (variants) from 20 to 30.
# Without reading its data, use metadata to show the shape of this subset.
subset = snp_reader[::2, 20:30]
print(f"Subset shape: {subset.shape} (individuals × SNPs)")

#  From the original SnpReader, read every value in chromosome 5
# into a numpy array.
val3 = snp_reader[:,snp_reader.pos[:,0] == 5].read().val
print(val3.shape)
val3

[['POP1' '0']
 ['POP1' '12']
 ['POP1' '44']
 ['POP1' '58']
 ['POP1' '65']]
['1_12' '1_34' '1_10' '1_35' '1_28']
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23.]


array([[0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 2., 0.]])

Subset shape: (150, 10) (individuals × SNPs)
(300, 43)


array([[0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
# Create a new snp_reader that re-orders and subsets the individuals
# and that leaves out chromosome 5.
iids_from_pheno_file = [["POP1", "65"], ["POP1", "44"], ["POP1", "0"]]
new_file_reader = snp_reader[snp_reader.iid_to_index(iids_from_pheno_file),snp_reader.pos[:,0] != 5]
print(f"# of individuals that will be read: {new_file_reader.iid_count}")
print(f"Note: not chrom 5 {np.unique(new_file_reader.pos[:,0])}")
# read just this data
val4 = new_file_reader.read().val
print(val4.shape)

# of individuals that will be read: 3
Note: not chrom 5 [ 1.  2.  3.  4.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 23.]
(3, 972)
(3, 972)


In [28]:
# Convert PySnpTools BED data to xarray Dataset with full genomic metadata
import xarray as xr

# Create xarray Dataset using numeric indexes (avoids MultiIndex complexity and ensures Zarr compatibility)
xarray_ds = xr.Dataset(
    {
        "genotypes": (["individual", "snp"], snp_reader.read().val)
    },
    coords={
        "individual": range(snp_reader.iid_count),
        "fid": (["individual"], [fid for fid, _iid in snp_reader.read().iid]),  # Family IDs
        "iid": (["individual"], [iid for _fid, iid in snp_reader.read().iid]),  # Individual IDs
        
        "snp": range(snp_reader.sid_count),
        "sid": (["snp"], snp_reader.sid),                                                 # SNP IDs
        "chromosome": (["snp"], np.nan_to_num(snp_reader.pos[:, 0], nan=0).astype(int)),  # Chromosome (NaN→0)
        "cm_position": (["snp"], snp_reader.pos[:, 1]),                                   # Genetic position
        "bp_position": (["snp"], np.nan_to_num(snp_reader.pos[:, 2], nan=0).astype(int)), # Physical position (NaN→0)
    },
    attrs={
        "description": "Genotype data from PySnpTools BED file",
        "encoding": "0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=missing",
        "source": url,
    }
)

display(xarray_ds)

<xarray.Dataset> Size: 3MB
Dimensions:      (individual: 300, snp: 1015)
Coordinates:
  * individual   (individual) int64 2kB 0 1 2 3 4 5 ... 294 295 296 297 298 299
    fid          (individual) <U4 5kB 'POP1' 'POP1' 'POP1' ... 'POP1' 'POP1'
    iid          (individual) <U4 5kB '0' '12' '44' ... '2987' '2998' '2999'
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
    sid          (snp) <U10 41kB '1_12' '1_34' '1_10' ... '23_49' '23_2' '23_3'
    chromosome   (snp) int64 8kB 1 1 1 1 1 1 1 1 1 ... 23 23 23 23 23 23 23 23
    cm_position  (snp) float64 8kB 0.008008 0.02302 0.07007 ... 0.9479 0.995
    bp_position  (snp) int64 8kB 0 1 4 6 7 8 9 10 11 ... 44 46 47 48 49 50 51 52
Data variables:
    genotypes    (individual, snp) float64 2MB 0.0 0.0 1.0 0.0 ... 0.0 2.0 0.0
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       https://raw.githubusercontent.com/fastlmm/FaST-LMM/refs/hea...

In [29]:
# Import zarr explicitly and save to Zarr format
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='zarr') # OK to ignore unicode warnings

# Save to Zarr format for efficient storage and access
zarr_path = "all_chr.maf0.001.N300.zarr"
xarray_ds.to_zarr(zarr_path, mode='w')

# Verify by loading back lazily
zarr_ds = xr.open_zarr(zarr_path)
display(zarr_ds)


<xarray.Dataset> Size: 3MB
Dimensions:      (snp: 1015, individual: 300)
Coordinates:
    bp_position  (snp) int64 8kB ...
    chromosome   (snp) int64 8kB ...
    cm_position  (snp) float64 8kB ...
    fid          (individual) <U4 5kB ...
    iid          (individual) <U4 5kB ...
  * individual   (individual) int64 2kB 0 1 2 3 4 5 ... 294 295 296 297 298 299
    sid          (snp) <U10 41kB ...
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
Data variables:
    genotypes    (individual, snp) float64 2MB ...
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       https://raw.githubusercontent.com/fastlmm/FaST-LMM/refs/hea...

In [30]:
# read all genotype data from the zarr file
zarr_ds.genotypes.values  # Accessing the genotype data directly

array([[0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 2., 0.]])

In [31]:
# Create a new zarr subset reader for every second individual
# and SNPs (variants) from 20 to 30.
zarr_subset = zarr_ds.genotypes[::2, 20:30]
zarr_subset.shape

(150, 10)

In [32]:
# On the original zarr_ds:
# List the first 5 individual (sample) ids, the first 5 SNP (variant) ids,
# and every unique chromosome. Then, read every value in chromosome 5.
print(zarr_ds.iid[:5].values)
print(zarr_ds.sid[:5].values)
print(np.unique(zarr_ds.chromosome))
# Compute the (Dask) boolean mask before using it for indexing
chr5_mask = (zarr_ds.chromosome == 5).compute()
zarr_val3 = zarr_ds.genotypes[:, chr5_mask].values
print(zarr_val3.shape)
print(zarr_val3)

['0' '12' '44' '58' '65']
['1_12' '1_34' '1_10' '1_35' '1_28']
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
(300, 43)
[[0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [33]:
# Create a new zarr reader that re-orders and subsets the individuals and that leaves out chromosome 5.
import pandas as pd

iids_from_pheno_file = [["POP1", "65"], ["POP1", "44"], ["POP1", "0"]]

# Need to do tricks to efficiently REORDER (and subset) the Dataset.
# Create pandas MultiIndex for efficient lookup
zarr_iids = pd.MultiIndex.from_arrays([zarr_ds.fid.values, zarr_ds.iid.values])
pheno_iids = pd.MultiIndex.from_tuples(iids_from_pheno_file)
indices = zarr_iids.get_indexer(pheno_iids)

print(f"Individual indices in phenotype file order: {indices}")

# Do subsetting in one line: individuals first (named), then SNPs (boolean)
new_zarr = zarr_ds.isel(individual=indices).sel(snp=zarr_ds.chromosome != 5)
print(f"# of individuals: {new_zarr.sizes['individual']}")
print(f"Chromosomes: {np.unique(new_zarr.chromosome)}")
display(new_zarr)

# Read just this data
val4_zarr = new_zarr.genotypes.values
print(f"Shape: {val4_zarr.shape}")

Individual indices in phenotype file order: [4 2 0]
# of individuals: 3
Chromosomes: [ 1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


<xarray.Dataset> Size: 93kB
Dimensions:      (snp: 972, individual: 3)
Coordinates:
    bp_position  (snp) int64 8kB ...
    chromosome   (snp) int64 8kB 1 1 1 1 1 1 1 1 1 ... 23 23 23 23 23 23 23 23
    cm_position  (snp) float64 8kB ...
    fid          (individual) <U4 48B 'POP1' 'POP1' 'POP1'
    iid          (individual) <U4 48B '65' '44' '0'
  * individual   (individual) int64 24B 4 2 0
    sid          (snp) <U10 39kB ...
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
Data variables:
    genotypes    (individual, snp) float64 23kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 0.0
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       https://raw.githubusercontent.com/fastlmm/FaST-LMM/refs/hea...

Shape: (3, 972)


In [34]:
from pathlib import Path
import bio2zarr.plink
from pysnptools.util import example_file

# Get local path to the example BED file
bed_path = example_file("tests/datasets/all_chr.maf0.001.N300.*")
base_path = str(Path(bed_path).with_suffix(''))
zarr_output = "plink_data.zarr"

if not Path(zarr_output).exists():
    bio2zarr.plink.convert(base_path, zarr_output)
bio2zarr_ds = xr.open_zarr(zarr_output)
bio2zarr_ds

<xarray.Dataset> Size: 2MB
Dimensions:               (variants: 1015, samples: 300, ploidy: 2,
                           contigs: 23, region_index_values: 24,
                           region_index_fields: 6, alleles: 2)
Dimensions without coordinates: variants, samples, ploidy, contigs,
                                region_index_values, region_index_fields,
                                alleles
Data variables:
    call_genotype         (variants, samples, ploidy) int8 609kB ...
    call_genotype_mask    (variants, samples, ploidy) bool 609kB ...
    call_genotype_phased  (variants, samples) bool 304kB ...
    contig_id             (contigs) object 184B ...
    region_index          (region_index_values, region_index_fields) int32 576B ...
    sample_id             (samples) object 2kB ...
    variant_allele        (variants, alleles) object 16kB ...
    variant_contig        (variants) int8 1kB ...
    variant_id            (variants) object 8kB ...
    variant_id_mask       (variants) bool 1kB ...
    variant_length        (variants) int8 1kB ...
    variant_position      (variants) int32 4kB ...
Attributes:
    source:            bio2zarr-0.1.6
    vcf_zarr_version:  0.4

In [35]:
# Compare bio2zarr dataset shape with PySnpTools shape
print(f"bio2zarr samples: {bio2zarr_ds.sizes['samples']}, variants: {bio2zarr_ds.sizes['variants']}")


bio2zarr samples: 300, variants: 1015


In [36]:
import numpy as np

# List the first 5 sample ids, the first 5 variant ids,
# and every unique chromosome from the bio2zarr dataset.
print("bio2zarr sample IDs (first 5):", bio2zarr_ds.sample_id[:5].values)
print("bio2zarr variant IDs (first 5):", bio2zarr_ds.variant_id[:5].values)

# Get unique chromosome names directly from contig_id
print("bio2zarr unique chromosomes:", np.unique(bio2zarr_ds.contig_id))


bio2zarr sample IDs (first 5): ['0' '12' '44' '58' '65']
bio2zarr variant IDs (first 5): ['1_12' '1_34' '1_10' '1_35' '1_28']
bio2zarr unique chromosomes: ['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22'
 '23' '3' '4' '5' '6' '7' '8' '9']


In [37]:
# Read all genotype data from bio2zarr dataset to an in-memory xarray Dataset
# that wraps a numpy array with genomic metadata.
bio2zarr_all = bio2zarr_ds.load()  # Load all data into memory
display(bio2zarr_all)

# Create the numpy array of genomic values suitable for GWAS
display(np.where(bio2zarr_all.call_genotype_mask.values.any(axis=-1).T, np.nan, bio2zarr_all.call_genotype.values.sum(axis=-1).T).astype(float))

<xarray.Dataset> Size: 2MB
Dimensions:               (variants: 1015, samples: 300, ploidy: 2,
                           contigs: 23, region_index_values: 24,
                           region_index_fields: 6, alleles: 2)
Dimensions without coordinates: variants, samples, ploidy, contigs,
                                region_index_values, region_index_fields,
                                alleles
Data variables:
    call_genotype         (variants, samples, ploidy) int8 609kB 0 0 0 ... 0 0 0
    call_genotype_mask    (variants, samples, ploidy) bool 609kB False ... False
    call_genotype_phased  (variants, samples) bool 304kB False False ... False
    contig_id             (contigs) object 184B '1' '2' '3' ... '21' '22' '23'
    region_index          (region_index_values, region_index_fields) int32 576B ...
    sample_id             (samples) object 2kB '0' '12' '44' ... '2998' '2999'
    variant_allele        (variants, alleles) object 16kB '1' '2' ... '1' '2'
    variant_contig        (variants) int8 1kB 0 0 0 0 0 0 ... 22 22 22 22 22 22
    variant_id            (variants) object 8kB '1_12' '1_34' ... '23_2' '23_3'
    variant_id_mask       (variants) bool 1kB False False False ... False False
    variant_length        (variants) int8 1kB 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    variant_position      (variants) int32 4kB 0 1 4 6 7 8 ... 47 48 49 50 51 52
Attributes:
    source:            bio2zarr-0.1.6
    vcf_zarr_version:  0.4

array([[0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 2., 0.]])

In [38]:
# Create a new bio2zarr subset for every second individual and SNPs (variants) from 20 to 30.
# Without reading its data, use metadata to show the shape of this subset.
bio2zarr_subset = bio2zarr_ds.isel(samples=np.s_[::2], variants=np.s_[20:30])
print(f"Subset shape: samples={bio2zarr_subset.sizes['samples']}, variants={bio2zarr_subset.sizes['variants']}")

Subset shape: samples=150, variants=10


In [39]:
# From the original bio2zarr dataset, read every value in chromosome 5
# into a numpy array (and convert to GWAS format).
# Create boolean mask for chromosome 5 and subset in one line
chrom5_subset = bio2zarr_ds.isel(variants=(bio2zarr_ds.contig_id.values[bio2zarr_ds.variant_contig.values] == "5"))
chrom5_gwas = np.where(chrom5_subset.call_genotype_mask.values.any(axis=-1).T, 
                       np.nan, 
                       chrom5_subset.call_genotype.values.sum(axis=-1).T).astype(float)
print(f"Chromosome 5 GWAS format shape: {chrom5_gwas.shape}")
display(chrom5_gwas)

Chromosome 5 GWAS format shape: (300, 43)


array([[0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
import pandas as pd

# Create a new bio2zarr reader that re-orders and subsets the individuals
# and that leaves out chromosome 5.
iids_from_pheno_file = [["POP1", "65"], ["POP1", "44"], ["POP1", "0"]]

# Ignore the family id and create indices for efficient lookup
indices = pd.Series(range(len(bio2zarr_ds.sample_id)),
                     index=bio2zarr_ds.sample_id.values).loc[
                         (iid for _fid, iid in iids_from_pheno_file)].values

# Do subsetting: individuals first (by indices), then exclude chromosome 5 in one operation
new_bio2zarr = bio2zarr_ds.isel(
    samples=indices,
    variants=(bio2zarr_ds.contig_id.values[bio2zarr_ds.variant_contig.values] != "5")
)

print(f"Individual sample IDs: {new_bio2zarr.sample_id.values}")
print(f"Unique chromosomes: {np.unique(bio2zarr_ds.contig_id.values[new_bio2zarr.variant_contig.values])}")
display(new_bio2zarr)

Individual sample IDs: ['65' '44' '0']
Unique chromosomes: ['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22'
 '23' '3' '4' '6' '7' '8' '9']


<xarray.Dataset> Size: 45kB
Dimensions:               (variants: 972, samples: 3, ploidy: 2, contigs: 23,
                           region_index_values: 24, region_index_fields: 6,
                           alleles: 2)
Dimensions without coordinates: variants, samples, ploidy, contigs,
                                region_index_values, region_index_fields,
                                alleles
Data variables:
    call_genotype         (variants, samples, ploidy) int8 6kB 0 0 0 0 ... 0 0 0
    call_genotype_mask    (variants, samples, ploidy) bool 6kB False ... False
    call_genotype_phased  (variants, samples) bool 3kB False False ... False
    contig_id             (contigs) object 184B '1' '2' '3' ... '21' '22' '23'
    region_index          (region_index_values, region_index_fields) int32 576B ...
    sample_id             (samples) object 24B '65' '44' '0'
    variant_allele        (variants, alleles) object 16kB '1' '2' ... '1' '2'
    variant_contig        (variants) int8 972B 0 0 0 0 0 0 ... 22 22 22 22 22 22
    variant_id            (variants) object 8kB '1_12' '1_34' ... '23_2' '23_3'
    variant_id_mask       (variants) bool 972B False False False ... False False
    variant_length        (variants) int8 972B 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    variant_position      (variants) int32 4kB 0 1 4 6 7 8 ... 47 48 49 50 51 52
Attributes:
    source:            bio2zarr-0.1.6
    vcf_zarr_version:  0.4

| Feature                | PySnpTools         |  simple Xarray/Zarr           | bio2zarr              |
|------------------------|--------------------|-----------------------|-----------------------|
| schema(s)              | fixed              | dynamic               | dynamic               |
| lazy loading           | yes                | yes                   | yes                   |
| full slicing & dicing  | yes (Numpy-like)   | yes (Pandas/Dask-like)| yes (Pandas/Dask-like) |
| compressed             | no                 | yes                   | yes (e.g. 40 times smaller) |
| in memory version      | `SnpData`          | `Xarray`              | `Xarray`              |
| random access          | yes                | yes                   | yes                   |
| index iid  | yes       | yes, external w/ Pandas | yes, external w/ Pandas no FID        |
| genotype encoding      | 0,1,2,NaN          | 0,1,2,NaN             | More general VCF-style|